In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

import pickle
import random
from string import punctuation

import gensim
from gensim import corpora

import spacy
from spacy.lang.en import English

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/katieta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# tokenizing/lemma-tizing stuff

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
#         elif token.orth_.startswith('#'):
#             lda_tokens.append('HASHTAG')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

def get_text_data(corpus, token_len=20):
    text_data = []
    for i in range(len(corpus)):
        if i % 20000 == 0:
            print("{} / {}".format(i, len(corpus)))
#             if i > 0:
#                 print(text_data[i - 1])
        try:
            tokens = prepare_text_for_lda(corpus[i])
            while len(tokens) < token_len:
                tokens.append('STOP')
                
            if len(tokens) > token_len:
                tokens = tokens[:token_len - 1]
                tokens.append('STOP')
                text_data.append(tokens)
            else:
                text_data.append(tokens)
        except Exception as e:
            print(e)
    print("finished")
    return text_data

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1
# print(stoplist)
en_stop = set(nltk.corpus.stopwords.words('english'))
# print(stoplist)

def get_dict_corpus(text_data):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2idx(text) for text in text_data]
    return dictionary, corpus

In [3]:
dem_tweets = pd.read_csv('../data/tweet_data/dem_tweets_cleaned.csv', index_col=False) # dem tweets w/ uncontested candidates removed
dem_tweets['party'] = [0 for i in range(len(dem_tweets))]
rep_tweets =pd.read_csv('../data/tweet_data/rep_tweets_cleaned.csv', index_col=False) # rep tweets w/ uncontested candidates removed
rep_tweets['party'] = [0 for i in range(len(rep_tweets))]

all_tweets = pd.read_csv('../data/tweet_data/tweets_with_topics.csv', index_col=False) # all tweets with labeled topics



In [4]:
all_tweets_no_topics = pd.read_csv('../data/tweet_data/all_tweets_cleaned.csv', index_col=False)


In [5]:
print(all_tweets_no_topics[all_tweets_no_topics['tweet_idx'] == 300000].stripped_text.values)
print(all_tweets[all_tweets['tweet_idx'] == 300000].stripped_text.values)

['our volunteers are are making calls for #gimmefitz all over the state. these knoxville supporters had a great time talking to their neighbors about our candidate fitz!']
['our volunteers are are making calls for #gimmefitz all over the state. these knoxville supporters had a great time talking to their neighbors about our candidate fitz!']


In [6]:
all_tweets.head(3)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax,fake_news,environment,sexual_assault,gender,women,lgbt,freedom,trump
0,0,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
1,1,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
2,2,here is an important notice concerning christo...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0


In [7]:
gun_control_keywords = [' gun ', 'gun ', 'gun control', 'guncontrol', 'gun reform', 'gunreformnow', 'nra'
                           ,'2nd amendment',  'assault weapon', '2a', 'firearm', 'tree of life', 'columbine'
                            , 'marchforourlives', 'shotgun', 'second amendment', 'gunsense', 'gunsensenow',
                       'momsdemand', 'gunviolence', 'wearorange', 'gunreform', 'semi-automatic', 'assault weapon'
                       , 'assault-type weapons']

healthcare_keywords = ['healthcare', 'health care', 'medicare', 'medicaid', 'health-care', 'affordable care act', 'aca ']

abortion_keywords = ['prolife', 'pro-life', 'pro life', 'prochoice', 'pro-choice', 'pro choice', 'reproductive'
                     , 'abortion', 'roe45', ' roe ']

immigration_keywords = ['immigration', 'borders', 'secure border', 'build wall', 'build a wall', 'build the wall'
                       , 'buildthewall', 'immigrant']

kavanaugh_keywords = ['kavanaugh']
education_keywords = ['education']
jobs_keywords = ['jobs', 'wages', 'workers']
tax_keywords = ['taxes', 'tax cut']
fake_news_keywords = ['fake news']
environment_keywords = ['global warming', 'plastic pollution'
                       , 'renewable energy', 'earthday', 'coastline', 
                        'renewable', 'solar', 'climatechange', 'carbon', 'fossil'
                       , 'nuclear plant']
women_keywords = ['women\'s rights', 'sexual assault', 'metoo']
lgbt_keywords = ['lgbt', 'transgender']
daca_keywords = ['daca']
freedom_keywords = ['freedom']
trump_keywords = ['trump', 'the president', 'potus']

def label_topic(tweet, keywords):
    for kw in keywords:
        if kw in tweet:
            return 1
    return 0

def re_label_topic(df, topic, keywords):
    for i, r in df.iterrows():
        if r[topic] == 0:
            r[topic] = label_topic(r['stripped_text'], keywords)
    
# all_tweets['gun_control'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, gun_control_keywords))
# all_tweets['health_care'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, healthcare_keywords))
# all_tweets['abortion'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, abortion_keywords))
# all_tweets['immigration'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, immigration_keywords))
# all_tweets['kavanaugh'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, kavanaugh_keywords))
# all_tweets['education'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, education_keywords))
# all_tweets['jobs'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, jobs_keywords))
# all_tweets['tax'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, tax_keywords))
# all_tweets['fake_news'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, fake_news_keywords))
# all_tweets['environment'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, environment_keywords))
# all_tweets['women'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, women_keywords))
# all_tweets['lgbt'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, lgbt_keywords))
# all_tweets['freedom'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, freedom_keywords))
# all_tweets['trump'] = all_tweets['stripped_text'].map(lambda x: label_topic(x, trump_keywords))


# for k in environment_keywords:
#     all_tweets.loc[all_tweets['stripped_text'].str.contains(k), 'environment'] = 1


In [8]:
# all_tweets = all_tweets.drop(labels=['trupm'], axis=1)
all_tweets.head(5)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax,fake_news,environment,sexual_assault,gender,women,lgbt,freedom,trump
0,0,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
1,1,with corruption being one of the biggest conce...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
2,2,here is an important notice concerning christo...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
3,3,i believe that we need to find real solutions ...,0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0
4,4,"it's time to turn the page on alabama's past, ...",0,False,0,0,0,0,0,0,0,0,0,False,False,False,0,0,0


# START HERE 

In [9]:
topic = 'jobs'
# all_tweets = all_tweets_topics

print(len(all_tweets[all_tweets[topic] == 1]))
topic_tweets = pd.concat([all_tweets[all_tweets[topic] == 1], all_tweets[all_tweets[topic] == 0].sample(len(all_tweets[all_tweets[topic] == 1]))])
print(len(topic_tweets))
full_tweets = pd.concat([topic_tweets, all_tweets[~all_tweets['tweet_idx'].isin(topic_tweets['tweet_idx'].unique())]])

print(len(all_tweets))
print(len(full_tweets))
# 

14195
28390
557549
557549


In [10]:
for t in topic_tweets[topic_tweets[topic] == 1].sample(100)['stripped_text'].values:
    print(t, '\n')

i'm looking forward to fighting for access to quality, affordable #healthcare, a tax code that builds our economy from the middle out, protecting social security & medicare, & an education system that creates a path for our kids to succeed. #il13 

talked w/the danville chamber of commerce this morning about how i will take my small business experience to congress to fight for them. small businesses prosper our state & we must get the government out of their way to allow them to create more jobs. #in04 #hardworkinghoosier 

pyrolysis! and yes! we can tackle climate change and create jobs right here. :) 

right to work laws have decimated the stability of keeping a good job, and being able to buy a home. the housing market can never recover, if citizens can't feel confident they can have a steady income, or wages to support a mortgage. #fightfor15 #everyonedeservesahome  

great way to end the week with @betoorourke talking about ways to improve texas for all texans. starting right here

In [11]:
# tokenize data 

model_token_data = get_text_data(full_tweets['stripped_text'].values)
# dictionary, corpus = get_dict_corpus(model_token_data)
# print(len(dictionary))
X = [{w: 1 for w in x} for x in model_token_data]
print(X[0])
dv = DictVectorizer()
X = dv.fit_transform(X)

print(X.shape)
# print(X.todense())
names = dv.get_feature_names()

0 / 557549
20000 / 557549
40000 / 557549
60000 / 557549
80000 / 557549
100000 / 557549
120000 / 557549
140000 / 557549
160000 / 557549
180000 / 557549
200000 / 557549
220000 / 557549
240000 / 557549
260000 / 557549
280000 / 557549
300000 / 557549
320000 / 557549
340000 / 557549
360000 / 557549
380000 / 557549
400000 / 557549
420000 / 557549
440000 / 557549
460000 / 557549
480000 / 557549
500000 / 557549
520000 / 557549
540000 / 557549
finished
{'here': 1, 'SCREEN_NAME': 1, 'alabama': 1, 'future': 1, 'economic': 1, 'sustainability': 1, 'create': 1, 'apprenticeship': 1, 'opportunity': 1, 'ensure': 1, 'every': 1, 'worker': 1, 'earn': 1, 'livable': 1, 'defend': 1, 'STOP': 1}
(557549, 122474)


In [12]:
print(len(topic_tweets))
topic_subset = X[:len(topic_tweets)]

X_train, X_test, y_train, y_test = train_test_split(topic_subset, topic_tweets[topic].values[:len(topic_tweets)], test_size=0.1)

new_file = True
if os.path.isfile('pickles/{}_logistic_regr.pkl'.format(topic)):
    with open('pickles/{}_logistic_regr.pkl'.format(topic), 'rb') as f:
        clf = pickle.load(f)
    new_file = False
else:
    clf = LogisticRegression()
    
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

# save model
with open('pickles/{}_logistic_regr.pkl'.format(topic), 'wb') as f:
    pickle.dump(clf, f)

for w, n in sorted(zip(clf.coef_[0], names), key=lambda e:e[0], reverse=True):
    print(w, n)

28390
7.446260623057823 worker
6.142633147491109 wages
5.070092205105721 taxcutsandjobsact
4.740338433051931 coworkers
3.8034416197527143 create
3.677130042630873 jobsnotmobs
3.532683696408938 steelworker
3.109013296441656 nvjobs
3.0548074763943314 overseas
2.8886443274335494 unemployment
2.87566926789356 manufacturing
2.766382191094764 pay
2.723931727226259 unionstrong
2.7122496859460825 outsource
2.616465523834459 caseworker
2.6118053902854057 taxcutandjobsact
2.6050784309661936 guarantee
2.5606787355977736 jobsreport
2.519677849664651 farmworkers
2.48820964534829 jobsjobsjobs
2.4680891610866866 steeleworkers
2.42506004869473 pete4jobs
2.3158572436057723 internationalworkersday
2.191776704559461 automation
2.150567644227174 infrastructure
2.0771683673070904 workersmemorialday
2.02773338030769 retail
2.0177753177052806 economy
1.9078144757776976 jobsguarantee
1.891747493159473 range
1.8563855134411436 collar
1.8378162829743816 diplomat
1.8352344558019917 export
1.8337361133718728 5000

0.6129470711085495 tracy
0.6127161989691973 americanworkers
0.6127161989691973 quasiboss
0.6116109105448129 agenda
0.6115606854185541 solidarity
0.6114659468964257 flourish
0.6109482888888432 raidernation
0.6105817974366051 connection
0.6100059674821412 misen
0.6096480411687416 purposeful
0.6093123538539411 pepper
0.6093013854575603 qualification
0.6090658602986461 greennewdeal
0.6088486839166413 oregonian
0.6086746483933606 hutchinson
0.6076881724245565 solid
0.6073625984622594 worthywages
0.6073212584687994 undercut
0.6071318698828597 dissect
0.6070127499043512 idgop
0.6057129286333022 hardiman
0.6053980543704445 single
0.6043565331590609 smell
0.6041856435396131 divisiveness
0.6039345686598326 august
0.603902737917146 whackjobs
0.6038873132905556 lifelonglearnerlottery
0.6038873132905556 qualityjobs
0.6038062223933757 battlefield
0.6036866430426379 pocketbook
0.6036159160698635 increasingly
0.6030710776722854 beautifulcleancoal
0.6026586548089558 teamkopser
0.6025610725708905 justif

0.4551640507978957 endangerment
0.45486455528121117 oppose
0.4548601373374606 pettinato
0.454837150960193 steam
0.4547735640445213 spartanburg
0.4546713803524212 innovate
0.45462900129609973 mahoning
0.45440872028106644 careless
0.45434207085311473 servicemembers
0.45389015429902124 maneuver
0.4534418795519831 latinosvote
0.45319073354103795 11,459
0.4531170160478256 dissapointment
0.4530074972190221 segment
0.45250518689091956 lebron
0.4524794019982409 innovation
0.4524467914144413 invisible
0.45239696061357826 ourkids
0.4521819612622779 hormone
0.4521670755835246 impede
0.45206204977544057 maybe
0.45127796214528004 complete
0.4512040738366822 envision
0.45113124112260833 taxmoney
0.4510171919181014 trickle
0.4509533942210034 225,000
0.4507877435106891 nhpolitics
0.4507828892690814 making
0.45063678802885404 concord
0.45058132740985607 betteroffnow
0.45054985997564767 campaignworkersguild
0.450422552618594 treadmill
0.4502766194402117 ihaveadream
0.4502766194402117 marchonwashington
0

0.3518705488974112 conservation
0.35182108381907096 systemic
0.3517565874135044 champion
0.3517207045919814 standwithworkers
0.35137787115883334 whop
0.3511819448587219 journal
0.35093956179816216 pa-15
0.3507485644922973 shipping
0.3507350496683107 equalrights
0.3503602784858117 cms%20sharing%20button&utm_medium
0.3503602784858117 ctpolilitics
0.3503602784858117 desktop&utm_source
0.3497957413078442 cordray
0.3496287458618419 speaker
0.34950088992589223 15/17
0.34950088992589223 equating
0.3494647063576469 oh-12
0.349448522781095 oneoklahoma
0.34941907202485895 base
0.3491303421701164 communities-
0.34882387905613405 presentation
0.34881824694495234 unionworkers
0.3488021118567972 fraternal
0.3487846206119455 complicit
0.34853763088533907 capitol
0.34805097431845655 catch-22
0.34804344242536267 psychotic
0.34798483470055763 goldman
0.34798483470055763 sachs
0.3478705256173978 allofustogether
0.3477209760319568 rather
0.3471647888447555 socal
0.3471070361196305 travesty
0.3470946769548

0.2621211727244108 hagemanforgovernor
0.26185230162867995 ordinary
0.2616158399530488 mary2018
0.2610274770947925 taylor
0.26090764333666205 payback
0.2608376129206046 tying
0.2606953603123084 maria
0.2601797586694162 unnecessary
0.259728432092402 plainfield
0.2595488829184097 geauga
0.25944856874583144 2,468
0.2594436482382358 bordersecutity
0.2594436482382358 toughonimmigration
0.2591910255710568 windfall
0.2591768107014554 mirage
0.25894887465055744 badly
0.2588892535699471 allsup
0.25888565550867604 contentious
0.2588593267521075 europe
0.25880224531011303 mercy
0.2587139933349803 mail
0.25823973792272364 vitality
0.25813215372447845 fraser
0.25795663564039994 marriage
0.2578973047567997 throw
0.25765901549950254 syracuse
0.25740342143771905 broadbandinternetforall
0.25740342143771905 georgiapaperballots
0.25740342143771905 ruraloffice
0.25739718376660503 stranger
0.2573660193017596 greenville
0.25732704941628 gallup
0.25731210602976934 ny23forum
0.25729385357886936 tnleg
0.2571666

...

-0.13712678064820694 durum
-0.13715025980633172 kaineandelaine
-0.1372311388226306 cowardly
-0.13723165310793392 traumatic
-0.1373086709140931 faint
-0.13736318877823334 squelch
-0.13738983442452662 bad
-0.13740815977019197 huckabee
-0.13741957083583542 retirekaine
-0.13741957083583542 takebackvirginia
-0.13744549237391424 conservatism
-0.1374678277169923 ifdogsranforcongress
-0.1374678277169923 wilfred
-0.13747613957027893 investigator
-0.13753091553124794 theresa
-0.13755053411366197 payroll
-0.1375664828471884 catalano
-0.1375664828471884 hoagie
-0.1375664828471884 importing
-0.1375664828471884 paul
-0.13759028595608092 gregorio
-0.13760966254085438 ben@kanewforcongress.com
-0.13767615521004375 course
-0.13769540926636165 rockdale
-0.13773130102239214 california
-0.1378009972183915 cemetery
-0.13787041132356992 earlyvote
-0.13788754264812064 intel
-0.1379037915230792 teamwatkins
-0.13798792894010253 prophet
-0.13799280812003598 peopleoverprofit
-0.1380155985482483 developing
-0.1380

-0.18115466266401725 graber
-0.18115466266401725 treynor
-0.18125695133203695 electvets
-0.18136510183626509 reviewer
-0.18141187900572567 disorder
-0.18159673573729565 boomerang
-0.1817485214988754 france
-0.18183589704787137 letuslead
-0.18184090115631657 afghanistan
-0.1818703663303214 frosh
-0.18190927016935945 lorch
-0.18202841898326194 exceptionally
-0.18214303749572153 amount
-0.18218808232589986 douglass
-0.18220866183173037 eastside
-0.18220866183173037 tardeada
-0.1824052734227898 unsure
-0.18242174940611666 impend
-0.18242174940611666 tweetit
-0.18244862422324315 starting
-0.18245646853748262 constant
-0.1826269430214801 windmill
-0.1826498434654907 kitchen
-0.1829356111280284 submit
-0.1829616853562684 dcision18
-0.1830981412491555 spokesman
-0.1831057817075019 acutely
-0.1831057817075019 autonomy
-0.1831338620682802 collegerepublicans
-0.1831338620682802 ursinus
-0.1832653684613255 portray
-0.18331195780600615 curtain
-0.18332444265476477 flyer
-0.18340831498622 autocorrec

-0.24918619130552788 differ
-0.24919982643694738 oceanside
-0.2493285119039103 enhance
-0.24941312303457075 schoolteacher
-0.24950353805700137 resultsnotrhetoric
-0.24954609867554858 thankfully
-0.24968222370675539 35,000,000
-0.24970504955241 o'neill
-0.24976822437314508 midterms2018
-0.24987337632824422 meantime
-0.2498803831331289 rabun
-0.25000240458320017 revitalization
-0.2500576734391003 adequate
-0.25014705686327576 ruleoflaw
-0.2503029305364673 upcoming
-0.25031718606071424 maga2018
-0.25038571688806815 syndrome
-0.2504280804060007 comforter
-0.25054653699446977 stacy
-0.2505739873639025 impromptu
-0.2505739873639025 nick
-0.25060888106517887 testimonialtuesday
-0.25062167999898843 paxton
-0.25072861555833303 113,000
-0.25077475333148236 terminal
-0.2508227503160245 embassy
-0.2509997687188224 outraised
-0.2512666973906164 powerball
-0.25135492786776514 dipstick
-0.2516588615367827 falsely
-0.25171125721771126 tonite
-0.2518089932179298 getcovered
-0.2518229189369836 ribbon
-0

-0.3667434797169414 campaign2018
-0.3669591528814634 apo18
-0.3669856896702516 oppression
-0.36751956339633 urine
-0.367789528000343 politicize
-0.3680724311651075 frankly
-0.36833901022597915 teamtucker
-0.36835181380620524 townsend
-0.36837384934258854 addsinsulttoinjury
-0.36837384934258854 overpayment
-0.36842925979168883 nc-05
-0.3685653142419549 distinguish
-0.36892529875034075 pressing
-0.36900587042115685 broadcast
-0.36908278624937174 legislator
-0.36913033134821027 points-
-0.36931656322831075 appearance
-0.36952530310847614 flooding
-0.3700011335452565 bookmaker
-0.3700011335452565 practically
-0.3700537139754155 soften
-0.3701287717580095 asset
-0.37014704872618687 sherry
-0.3701669320258134 phyllis
-0.37045577704339877 airport
-0.37045853952987245 trustee
-0.3705646203544037 river
-0.37062808917480017 legislate
-0.3706497076458659 oprah
-0.3707513620802366 toilet
-0.37077912790975315 jeffersonville
-0.3710555598836422 prevail
-0.3712264290636006 congressional
-0.3718575824

-0.6351835002667703 polls
-0.6352814435502159 entitlement
-0.6353045289961144 inspiration
-0.6371785106994484 usual
-0.6380494864048152 although
-0.6382141072735776 progressherenow
-0.6386006809456289 okgov
-0.6386038415141043 honesty
-0.6397454339557194 basketball
-0.640134113399766 kalamazoo
-0.6404313226617911 turntexasblue
-0.6408120933871768 oakland
-0.6412685220028419 hawai`i
-0.6419033582923054 bhsil06forum
-0.6420294771432244 pick
-0.6420670761060157 shock
-0.6428518234792676 venezuela
-0.6428859918640628 mdga18
-0.6430287537211627 safely
-0.6445600812243397 tunisia
-0.6448331882271783 enthusiasm
-0.6456680450275317 species
-0.6457715922197445 asylum
-0.6460991162573543 intern
-0.6479250889810688 accountable
-0.6481036769383797 substance
-0.6484082843560833 depression
-0.6486182290676298 final
-0.6487093921647475 edawn
-0.6489769894681754 tellus
-0.6498363800604415 extent
-0.6507880831056192 noon105
-0.6509040079658438 firefighter
-0.6518121568101419 disease
-0.6525015191865469

In [107]:
c = 0
for i in y_train:
    if i == 1:
        c += 1 
print(c)
print(len(y_train))

31858
63720


In [108]:
score

0.9149717514124294

## Run Predictions

In [109]:
pred_probs = clf.predict_proba(X[len(topic_tweets) + 1:])
pred_probs[0]

array([0.99824829, 0.00175171])

In [110]:
predictions = clf.predict(X[len(topic_tweets) + 1:])
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [111]:
c = 0
t = 0
tw = []
below_threshold = 0
tweets = full_tweets['stripped_text'].values[len(topic_tweets) + 1:]
full_tweets_idx = full_tweets['tweet_idx'].values[len(topic_tweets) + 1:]
idx = []
missed_tweets = []
tweets_above_threshold = []
for i in range(len(pred_probs)):
    if pred_probs[i][1] >= 0.99:
        t += 1
        idx.append(full_tweets_idx[i])
        tweets_above_threshold.append(tweets[i])
        
    else:
        below_threshold += 1
        
print("num above threshold: ", len(tweets_above_threshold)) # total above threshold
print("below threshold: ", below_threshold)

num above threshold:  884
below threshold:  485864


In [112]:
for t in np.random.choice(tweets_above_threshold, min(len(tweets_above_threshold), 100)):
    print(t, '\n')

leading mexican presidential candidate obrador calls for mass exodus to united states when he wins election   via @gatewaypundit 

the economy is booming and our #americafirst agenda is benefiting all  hispanic unemployment has now reached lowest level in 45 years! yet @susieleenv wants to reverse that historic achievement and make america a failure again. vote tark! #nv03  

jaime, sophia, i had a great time hearing vice president @mike_pence rally supporters today in carson city. isabella and jack would have too but it was nap time! #nvgov 

breaking: @politifact #lieofyear goes to president 45.joins ranks of pittenger."-cc #nc09 #flipthe9th #countryoverparty #staywoke #calledtoserveothers #staywoke #dantheflipper #wherewasdanin2016 #windmillgate @ourrevolution @latinovictoryus @votolatino   

. @joenbc dear mr. president, it has become obvious that you don't have an ivy league education either. you may have gone to an ivy league school, but you have demonstrated that you didn't lear

In [129]:
# label the tweets above threshold with a 1
# all_tweets.loc[all_tweets['tweet_idx'].isin(idx), topic] = 1

all_tweets.to_csv('../data/tweet_data/tweets_with_topics.csv', index=False)

In [569]:
all_tweets.head(10)

,tweet_idx,stripped_text,party,gun_control,health_care,abortion,immigration,kavanaugh,education,jobs,tax
0,0,with corruption being one of the biggest conce...,0,0,0,0,0,0,0,0,0
1,1,with corruption being one of the biggest conce...,0,0,0,0,0,0,0,0,0
2,2,here is an important notice concerning christo...,0,0,0,0,0,0,0,0,0
3,3,i believe that we need to find real solutions ...,0,0,0,0,0,0,0,0,0
4,4,"it's time to turn the page on alabama's past, ...",0,0,0,0,0,0,0,0,0
5,5,there is hope for alabama. this inspiring vide...,0,0,0,0,0,0,0,0,0
6,6,live,0,0,0,0,0,0,0,0,0
7,7,join us for a special and important webcast co...,0,0,0,0,0,0,0,0,0
8,8,countryman isn't standing down...his fight con...,0,0,0,0,0,0,0,0,0
9,9,please join us for a very special 2018 alabama...,0,0,0,0,0,0,0,0,0


In [8]:
topics = ['gun_control', 'health_care', 'abortion', 'immigration', 'kavanaugh', 'education', 'jobs', 'tax']

unlabeled_tweets = all_tweets[(all_tweets['gun_control'] == 0) 
           & (all_tweets['health_care'] == 0)
          & (all_tweets['abortion'] == 0)
          & (all_tweets['immigration'] == 0)
          & (all_tweets['kavanaugh'] == 0)
          & (all_tweets['education'] == 0)
          & (all_tweets['jobs'] == 0)
          & (all_tweets['tax'] == 0)]

In [18]:
print(len(unlabeled_tweets))
print(len(unlabeled_tweets) / len(all_tweets))
print('\n')
for t in unlabeled_tweets.sample(10)['stripped_text'].values:
    print(t, '\n')

485470
0.8707216764804528


its great to be a new mexican  especially during green chile roasting season! i stopped by some stands in hatch to get my fix of fresh roasted chile! an important tradition, growing chile employs thousands and brings in hundreds of millions in revenue each year. #nmpol 

to the 1,177 brothers who lost their lives on the uss arizona and the 2,403 sailors, soldiers, marines and civilians who died at pearl harbor 76 years ago today, america will never forget your ultimate sacrifice. 

the campaign has already begun. sign up at  if you'd like to help! appreciate the support! 

i was speaking metaphorically about loving my neighbor in practical ways. but if you're asking about the lambs in that photo (from my family's farm), i'll be honest. (you deserve honesty.) they are mutton. while i deeply respect vegetarianism, i don't practice it. 

lots of doors still to knock on in here the final 10 days until the primary. fun and humbling to visit with people face-to-fa

In [102]:
len(unlabeled_tweets[unlabeled_tweets['stripped_text'].str.contains('corruption')])

1702

In [103]:
for t in unlabeled_tweets[unlabeled_tweets['stripped_text'].str.contains('corruption')].sample(100)['stripped_text'].values:
    print(t, '\n')

thank you to senator pierre from haiti and the delegation for their invitation today at the rotary club of bellaire. haiti is on the rise as they strive to eliminate corruption and create opportunity for its people. join our #armyofdavids!   

heartbreaking: sudan, world's last male northern white rhino, dies"  we must stop the multi-billion dollar industry that is wildlife trafficking. this industry thrives on corruption, funds terrorism, and results in the extinction of animals. @humanesociety" 

#crushcorruption #firefeinstein   

yeah there might be some major corruption going on here(?) 

my opponent @repmimiwalters sits on the house energy & commerce oversight committee, which has jurisdiction over jared & ivanka's alleged corruption. not only has she not called for a hearing, she's never once even publicly mentioned these disturbing reports. #wheresmimi 

joining @thedaveweinbaum today on his show! 11:15 pst we are gonna talk alabama corruption and election fraud!.. his link is 

In [32]:
len(all_tweets[all_tweets['abortion'] == 1])

2879